In [1]:
# Configura o path do projeto para consumir as libs do projeto
# Path para diretório raiz do projeto
import sys

sys.path.insert(0, sys.path[0].removesuffix("/src/jupyter"))
# Libs do projeto
import src.utils as utils
import pandas as pd
import numpy as np

# All necessary plotly libraries
import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

## **** Jupyter Notebook responsável por analisar graficamente os indicadores calculados previamente para cada Ticker (Ações ou FII's) ****

In [2]:
# Recupera base de dados com histórico de cotações de Ações e FII's baixadas previamente 
data_file_list = utils.get_data_files()

acoes_prob_cair = pd.DataFrame() # Inicializa lista que irá conter ações com indicação de queda
acoes_prob_subir = pd.DataFrame() # Inicializa lista que irá conter ações com indicação de alta
for data_file in data_file_list:
    data = pd.read_csv(data_file, sep=";") # Lê base de histórico de cotações do ticket correspondente
    data.dropna(inplace=True) # Limpa linhas vazias, caso exista
    if data.shape[0] > 0:
        data["previsao"] = "ESTAVEL" # Em princípio, marca todos os registros como "Estáveis", ou seja, sem indicativo de queda ou alta
        data["previsao"] = np.where(
            (data["adj_close"] > data["ema_close_200p"]) & (data["rsi"] > 70.0) & (data["volume"] > 0),
            "CAIR",
            data["previsao"],
        ) # Calcula previsão de queda com base nos indicadores EMA e RSI
        data["previsao"] = np.where(
            (data["adj_close"] < data["ema_close_200p"]) & (data["rsi"] < 30.0) & (data["volume"] > 0),
            "SUBIR",
            data["previsao"],
        ) # Calcula previsão de alta com base nos indicadores EMA e RSI
        data.to_csv(data_file, sep=";", index=False) # Grava base de dados de cada ticket com indicadores calculados

        if data.tail(1).previsao.values[0] == "CAIR": # Inclui ticket na lista, caso tenha ocorrido uma previsão de cair
            acoes_prob_cair = pd.concat([acoes_prob_cair, data.tail(1)])
        elif data.tail(1).previsao.values[0] == "SUBIR": # Inclui ticket na lista, caso tenha ocorrido uma previsão de subir
            acoes_prob_subir = pd.concat([acoes_prob_subir, data.tail(1)])

In [3]:
# Lista tickers com previsão de cair
print(f"Ações Probabilidade Cair: ")
acoes_prob_cair

Ações Probabilidade Cair: 


,s_open_time,open_time,ticker,open,high,low,close,adj_close,volume,ticker_type,ema_adj_close_200p,ema_adj_close_200p_diff,ema_close_200p,ema_close_200p_diff,rsi,previsao
3808,2024-02-26,1708905600000,ALPA4.SA,10.01,10.27,9.78,10.17,10.17,4585600,stock,9.75,4.34,9.75,4.32,71.21,CAIR
3439,2024-02-26,1708905600000,CIEL3.SA,5.33,5.49,5.33,5.43,5.43,29779900,stock,4.35,24.89,4.46,21.65,72.19,CAIR
5097,2024-02-26,1708905600000,UGPA3.SA,30.40,30.87,30.40,30.87,30.87,3410300,stock,22.42,37.68,22.50,37.23,70.04,CAIR
5864,2024-02-26,1708905600000,USIM5.SA,10.58,10.80,10.54,10.67,10.67,13929700,stock,8.02,32.97,8.08,32.00,79.66,CAIR
1276,2024-02-26,1708905600000,VBBR3.SA,25.69,26.08,25.52,25.90,25.90,6178100,stock,20.10,28.83,20.50,26.36,71.22,CAIR


In [4]:
# Lista tickers com previsão de subir
print(f"Ações Probabilidade Subir: ")
acoes_prob_subir

Ações Probabilidade Subir: 


,s_open_time,open_time,ticker,open,high,low,close,adj_close,volume,ticker_type,ema_adj_close_200p,ema_adj_close_200p_diff,ema_close_200p,ema_close_200p_diff,rsi,previsao
4502,2024-02-26,1708905600000,AMER3.SA,0.56,0.56,0.50,0.50,0.50,41489100,stock,1.93,-74.16,1.94,-74.23,19.29,SUBIR
2767,2024-02-26,1708905600000,COGN3.SA,2.36,2.41,2.36,2.38,2.38,16307200,stock,2.85,-16.48,2.85,-16.48,28.18,SUBIR
4696,2024-02-26,1708905600000,GOLL4.SA,2.63,2.71,2.60,2.60,2.60,6738000,stock,7.33,-64.52,7.33,-64.52,26.87,SUBIR


In [5]:
filter_date = '2023-01-01'
int_date = int(pd.to_datetime(filter_date, format='%Y-%m-%d').to_datetime64().astype(int) / 1000000)
int_date

1672531200000

In [7]:
print('Graficos Ações Probabilidade Subir')
fig_list = []
for ticker in acoes_prob_subir.ticker.unique():
	ticker_type = data.tail()['ticker_type'].values[0]
	title = f'{ticker_type.upper()}: {ticker} - Probabilidade Subir'
	data = pd.read_csv(f"{utils.stocks_dir}/{ticker}.csv", sep=";")
	data = data[data['open_time'] > int_date]
	fig = make_subplots(rows=2, cols=1, row_heights=[0.7, 0.3], specs=[[{"secondary_y": True}], [{"secondary_y": True}]])
	
	# fig.add_trace(go.Candlestick(x=data['s_open_time'], open=data['open'], high=data['high'], low=data['low'], close=data['close'], name='Price'), row=1, col=1)
	fig.add_trace(go.Scatter(x=data['s_open_time'], y=data['close'], name='Close'), row=1, col=1)
	fig.add_trace(go.Scatter(x=data['s_open_time'], y=data['adj_close'], name='Adj Close'), row=1, col=1)
	fig.add_trace(go.Scatter(x=data['s_open_time'], y=data['ema_close_200p'], name='EMA Close 200p', line_shape='spline', line_smoothing=1.3), row=1, col=1)
	fig.add_trace(go.Scatter(x=data['s_open_time'], y=data['ema_adj_close_200p'], name='EMA Adj Close 200p', line_shape='spline', line_smoothing=1.3), row=1, col=1)

	fig.add_trace(go.Scatter(x=data['s_open_time'], y=data['rsi'], name='rsi', line_color='purple'), row=2, col=1, secondary_y=False)
	fig.add_trace(go.Bar(x=data['s_open_time'], y=data['volume'], name='volume', marker_color='red'), row=2, col=1, secondary_y=True)
	
	fig.add_hline(y=30, row=2, line_width=1, line_dash="dash", line_color='black')
	fig.add_hline(y=70, row=2, line_width=1, line_dash="dash", line_color='black')
	fig.add_hrect(y0=30, y1=70, line_width=0, fillcolor="blue", opacity=0.1, row=2)
	
	fig.update_layout(title_text=title, height=800, )
	fig.update(layout_xaxis_rangeslider_visible=False)
	fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "sun"])])

	fig_list.append({'fig': fig, 'title': f'{title}' })
	

for fig in fig_list:
	print(fig['title'])
	fig['fig'].show()

Graficos Ações Probabilidade Subir
STOCK: AMER3.SA - Probabilidade Subir


STOCK: COGN3.SA - Probabilidade Subir


STOCK: GOLL4.SA - Probabilidade Subir


In [8]:
print('Graficos Ações Probabilidade Cair')
fig_list = []
for ticker in acoes_prob_cair.ticker.unique():
	ticker_type = data.tail()['ticker_type'].values[0]
	title = f'{ticker_type.upper()}: {ticker} - Probabilidade Cair'
	data = pd.read_csv(f"{utils.stocks_dir}/{ticker}.csv", sep=";")
	data = data[data['open_time'] > int_date]
	fig = make_subplots(rows=2, cols=1, row_heights=[0.7, 0.3], specs=[[{"secondary_y": True}], [{"secondary_y": True}]])
	
	# fig.add_trace(go.Candlestick(x=data['s_open_time'], open=data['open'], high=data['high'], low=data['low'], close=data['close'], name='Price'), row=1, col=1)
	fig.add_trace(go.Scatter(x=data['s_open_time'], y=data['close'], name='Close'), row=1, col=1)
	fig.add_trace(go.Scatter(x=data['s_open_time'], y=data['adj_close'], name='Adj Close'), row=1, col=1)
	fig.add_trace(go.Scatter(x=data['s_open_time'], y=data['ema_close_200p'], name='EMA Close 200p', line_shape='spline', line_smoothing=1.3), row=1, col=1)
	fig.add_trace(go.Scatter(x=data['s_open_time'], y=data['ema_adj_close_200p'], name='EMA Adj Close 200p', line_shape='spline', line_smoothing=1.3), row=1, col=1)

	fig.add_trace(go.Scatter(x=data['s_open_time'], y=data['rsi'], name='rsi', line_color='purple'), row=2, col=1, secondary_y=False)
	fig.add_trace(go.Bar(x=data['s_open_time'], y=data['volume'], name='volume', marker_color='red'), row=2, col=1, secondary_y=True)
	
	fig.add_hline(y=30, row=2, line_width=1, line_dash="dash", line_color='black')
	fig.add_hline(y=70, row=2, line_width=1, line_dash="dash", line_color='black')
	fig.add_hrect(y0=30, y1=70, line_width=0, fillcolor="blue", opacity=0.1, row=2)
	
	fig.update_layout(title_text=title, height=800, )
	fig.update(layout_xaxis_rangeslider_visible=False)
	fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "sun"])])

	fig_list.append({'fig': fig, 'title': f'{title}' })
	

for fig in fig_list:
	print(fig['title'])
	fig['fig'].show()

Graficos Ações Probabilidade Cair
STOCK: ALPA4.SA - Probabilidade Cair


STOCK: CIEL3.SA - Probabilidade Cair


STOCK: UGPA3.SA - Probabilidade Cair


STOCK: USIM5.SA - Probabilidade Cair


STOCK: VBBR3.SA - Probabilidade Cair
